### Imports
Import useful modules and classes to perform the connection to MongoDB.

In [1]:
# Import os to manage paths
import os

# Import certifi to manage SSL certificates with TSL host
import certifi

# Import ConfigParser for handling config file
from configparser import ConfigParser

# Generate pretty outputs
from pprint import pprint

# For dates handling
from datetime import datetime, timedelta

# Import PyMongo useful classes
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

### Read the configuration file and load the credentials
Load sensible data from a configuration file.

In [2]:
# Read the configuration file with the credentials
config = ConfigParser()
config.read(os.path.join(os.getcwd(), 'config.ini'))

# Load the data to connect to MongoDB Atlas Cluster
user = config['MongoDB']['user']
password = config['MongoDB']['password']
uri = config['MongoDB']['uri']
db_name = config['MongoDB']['db_name']

### Connect to MongoDB Atlas Cluster
Try to connect to Atlas Cluster (AWS Cloud) and print the server info to ensure connection. If the output is a dict, the connection is ok.

In [3]:
# Generate the connection string for the client
CONNECTION_STRING = "mongodb+srv://{}:{}@{}/{}?retryWrites=true&w=majority"\
                    .format(user, password, uri, db_name)

# Connect to MongoDB Atlas Cluster
client = MongoClient(CONNECTION_STRING, 
                     server_api=ServerApi('1'),
                     tls=True,
                     tlsCAFile=certifi.where())

# Check if connection succeed
try:
    pprint(client.server_info())
except Exception:
    print("Unable to connect to the server.")

{'$clusterTime': {'clusterTime': Timestamp(1652118462, 2),
                  'signature': {'hash': b'Q\xdb1\xef\xf3]\x16\x90O\x0emL)]\x9fz'
                                        b'\x90\xc6\xc7\xc0',
                                'keyId': 7071822064142778372}},
 'allocator': 'tcmalloc',
 'bits': 64,
 'buildEnvironment': {'cc': '/opt/mongodbtoolchain/v3/bin/gcc: gcc (GCC) 8.5.0',
                      'ccflags': '-Werror -include mongo/platform/basic.h '
                                 '-fasynchronous-unwind-tables -ggdb -Wall '
                                 '-Wsign-compare -Wno-unknown-pragmas '
                                 '-Winvalid-pch -fno-omit-frame-pointer '
                                 '-fno-strict-aliasing -O2 -march=sandybridge '
                                 '-mtune=generic -mprefer-vector-width=128 '
                                 '-Wno-unused-local-typedefs '
                                 '-Wno-unused-function '
                                 '-Wno-

### NoSQL vs SQL Databases
In order to compare NoSQL (MongoDB) with SQL Databases, let's see the following chart which link concepts between each other:

| Relational concept (SQL) | MongoDB equivalent (NoSQL) |
| :-: | :-: |
| Database | Database |
| Tables | Collections |
| Rows | Documents |
| Index | Index |

Once understood, we can continue the notebook retrieving some info from the Cloud. Let's see an example of some CRUD (Create, Read, Update, Delete) operations over a Sample Dataset that Atlas provides.

In [4]:
# Acces to the DB "sample_analytics"
# For more info, visit: https://www.mongodb.com/docs/atlas/sample-data/sample-analytics/
db = client.get_database('sample_analytics')

# Once pointed the DB, lest get all the collections names contained
db.list_collection_names()

['customers', 'accounts', 'transactions']

### CRUD Operations with PyMongo

Let's visualize some info of the *transactions* collection and make some CRUD operations.

This collection contains transactions details for users. Each document contains an account id, a count of how many transactions are in this set, the start and end dates for transactions covered by this document, and a list of sub documents. Each sub document represents a single transaction and the related information for that transaction.

For more info, please visit: https://www.mongodb.com/docs/atlas/sample-data/sample-analytics/#std-label-analytics-transactions

Below you can see the usage of the comparision operators.

In [5]:
# Create a new object with the collection (table)
transactions = db.get_collection('transactions')

print('Usage of the comparision operators of PyMongo:')

# Let's see how many documents (rows) are there in this collection (table)
num_docs = transactions.estimated_document_count()
print('There are {} documents in the transactions collection.'.format(num_docs))

# ¿How many documents with 6 transactions are in the DB?
trans_count = 6
six_trans_count = transactions.count_documents(filter={'transaction_count': trans_count})
print('There are {} documents in the transactions collection with {} transactions.'\
      .format(six_trans_count, trans_count))

# Other way to get the same result is as following:
six_trans_count = transactions.count_documents(filter={'transaction_count': {'$eq': trans_count}})
print('There are {} documents in the transactions collection with {} transactions.'\
      .format(six_trans_count, trans_count))

# Now let's see how many documents with more than 6 transactions are in the DB
more_six_trans_count = transactions.count_documents(filter={'transaction_count': {'$gt': trans_count}})
print('There are {} documents in the transactions collection with more than {} transactions.'\
      .format(more_six_trans_count, trans_count))

# ¿How many documents with 6 or 12 transactions are in the DB?
range_trans_count = transactions.count_documents(filter={'transaction_count': {'$in': [trans_count, 2*trans_count]}})
print('There are {} documents in the transactions collection with {} to {} transactions.'\
      .format(range_trans_count, trans_count, 2*trans_count))


Usage of the comparision operators of PyMongo:
There are 1746 documents in the transactions collection.
There are 13 documents in the transactions collection with 6 transactions.
There are 13 documents in the transactions collection with 6 transactions.
There are 1641 documents in the transactions collection with more than 6 transactions.
There are 33 documents in the transactions collection with 6 to 12 transactions.


In [6]:
# Visualize one of the examples
six_trans_docs = transactions.find_one(filter={'transaction_count': trans_count})
pprint(six_trans_docs)

{'_id': ObjectId('5ca4bbc1a2dd94ee58161cdf'),
 'account_id': 794875,
 'bucket_end_date': datetime.datetime(2016, 9, 6, 0, 0),
 'bucket_start_date': datetime.datetime(1991, 12, 27, 0, 0),
 'transaction_count': 6,
 'transactions': [{'amount': 1197,
                   'date': datetime.datetime(2011, 12, 28, 0, 0),
                   'price': '12.7330024299341033611199236474931240081787109375',
                   'symbol': 'nvda',
                   'total': '15241.40390863112172326054861',
                   'transaction_code': 'buy'},
                  {'amount': 8797,
                   'date': datetime.datetime(2016, 6, 13, 0, 0),
                   'price': '46.53873172406391489630550495348870754241943359375',
                   'symbol': 'nvda',
                   'total': '409401.2229765902593427995271',
                   'transaction_code': 'buy'},
                  {'amount': 6146,
                   'date': datetime.datetime(2016, 8, 31, 0, 0),
                   'price': '32.11

Now, let's see the logical operators

In [7]:
print('Example usage of the logical operators of PyMongo:')

# Set a start/end date to perform the search
start = datetime(2000, 1, 1, 0, 0, 0)
end = datetime(2001, 1, 1, 0, 0, 0)

# Find one document with more than 6 transactions,
# and bucket_start_date between 1/1/2000 and 1/1/2001
logical = transactions.find_one({
    '$and': [{
                'transaction_count': {'$gte': trans_count}
             },
             {
                'bucket_start_date': {'$gte': start}
             },
             {
                'bucket_start_date': {'$lte': end}
             }
             ]
})

# Visualize first result
pprint(logical)

Example usage of the logical operators of PyMongo:
{'_id': ObjectId('5ca4bbc1a2dd94ee58161d85'),
 'account_id': 161714,
 'bucket_end_date': datetime.datetime(2017, 1, 8, 0, 0),
 'bucket_start_date': datetime.datetime(2000, 12, 9, 0, 0),
 'transaction_count': 35,
 'transactions': [{'amount': 3690,
                   'date': datetime.datetime(2016, 12, 30, 0, 0),
                   'price': '777.323567301198409040807746350765228271484375',
                   'symbol': 'goog',
                   'total': '2868323.963341422129360580584',
                   'transaction_code': 'buy'},
                  {'amount': 8245,
                   'date': datetime.datetime(2016, 1, 5, 0, 0),
                   'price': '110.360572720722842632312676869332790374755859375',
                   'symbol': 'nflx',
                   'total': '909922.9220823598375034180208',
                   'transaction_code': 'buy'},
                  {'amount': 978,
                   'date': datetime.datetime(2015, 3, 

### Create
Let's see an example of ho to insert new data in the DB, and visualize the *_id* generated for this new document.

In [8]:
# An example document to insert
document = {
    'account_id': 800000,
    'bucket_end_date': datetime.now() + timedelta(7),
    'bucket_start_date': datetime.now(),
    'transaction_count': 1,
    'transactions': [{
                      'amount': 1000,
                      'date': datetime.now(),
                      'price': '10.00',
                      'symbol': 'nvda',
                      'total': '10000.00',
                      'transaction_code': 'buy'
                      }]
}

# Insert the data
try: 
    insert = transactions.insert_one(document)
    print('The _id generated is: {}'.format(insert.inserted_id))
except Exception:
    print('An error has ocurred during the insert.')

The _id generated is: 627953bf45eb5273ac07bbdf


### Read
In the below examples, we can see how to fetch some results from the DB.

In [9]:
# If no filtering, the entire DB is fetched.
# In this example, limit to the first 3 documents.
# This is equivalent to SELECT * FROM transactions;
first_three_docs = transactions.find({}).limit(3)

# Let's see the results
for doc in first_three_docs:
    pprint(doc)

{'_id': ObjectId('5ca4bbc1a2dd94ee58161cb1'),
 'account_id': 443178,
 'bucket_end_date': datetime.datetime(2017, 1, 3, 0, 0),
 'bucket_start_date': datetime.datetime(1969, 2, 4, 0, 0),
 'transaction_count': 66,
 'transactions': [{'amount': 7514,
                   'date': datetime.datetime(2003, 9, 9, 0, 0),
                   'price': '19.1072802650074180519368383102118968963623046875',
                   'symbol': 'adbe',
                   'total': '143572.1039112657392422534031',
                   'transaction_code': 'buy'},
                  {'amount': 9240,
                   'date': datetime.datetime(2016, 6, 14, 0, 0),
                   'price': '24.1525632387771480580340721644461154937744140625',
                   'symbol': 'team',
                   'total': '223169.6843263008480562348268',
                   'transaction_code': 'buy'},
                  {'amount': 2824,
                   'date': datetime.datetime(2002, 12, 4, 0, 0),
                   'price': '21.046193

Now let's retrieve the same document, but except the *transactions* field

In [10]:
# To retrieve only specific data, the projection parameter must be used
first_three_docs = transactions.find(filter={}, projection={'transactions': False}).limit(3)

# Let's see the results
for doc in first_three_docs:
    pprint(doc)

{'_id': ObjectId('5ca4bbc1a2dd94ee58161cb1'),
 'account_id': 443178,
 'bucket_end_date': datetime.datetime(2017, 1, 3, 0, 0),
 'bucket_start_date': datetime.datetime(1969, 2, 4, 0, 0),
 'transaction_count': 66}
{'_id': ObjectId('5ca4bbc1a2dd94ee58161cb2'),
 'account_id': 716662,
 'bucket_end_date': datetime.datetime(2016, 12, 27, 0, 0),
 'bucket_start_date': datetime.datetime(1962, 5, 13, 0, 0),
 'transaction_count': 48}
{'_id': ObjectId('5ca4bbc1a2dd94ee58161cb3'),
 'account_id': 557378,
 'bucket_end_date': datetime.datetime(2016, 11, 6, 0, 0),
 'bucket_start_date': datetime.datetime(1990, 6, 11, 0, 0),
 'transaction_count': 56}


### Update
Now we can update the insert that we've done before. For example:

In [11]:
# Perform the update
try: 
    update = transactions.update_one(
        filter={'_id': insert.inserted_id}, # Previously created ID
        update={'$set': {'account_id': 900000}} # Previously was 'account_id': 800000
    )
    print('Number of documents that matched the filter: {}'.format(update.matched_count))
    print('Number of documents affected in the update: {}'.format(update.modified_count))
    print('_id of the documents affected in the update: {}'.format(update.upserted_id))
    print('Visualize the raw result: ')
    pprint(update.raw_result)
except Exception:
    print('Error during update.')

Number of documents that matched the filter: 1
Number of documents affected in the update: 1
_id of the documents affected in the update: None
Visualize the raw result: 
{'$clusterTime': {'clusterTime': Timestamp(1652118464, 1),
                  'signature': {'hash': b'\xd5OS|\xc8\xcf\xed\xc0'
                                        b'\x99\x10\xc1\xa5\x99Om\x8a'
                                        b'\xa5\x04\x8e;',
                                'keyId': 7071822064142778372}},
 'electionId': ObjectId('7fffffff0000000000000006'),
 'n': 1,
 'nModified': 1,
 'ok': 1.0,
 'opTime': {'t': 6, 'ts': Timestamp(1652118464, 1)},
 'operationTime': Timestamp(1652118464, 1),
 'updatedExisting': True}


### Delete
Finally, let's see how to delete the document that we generate before.

In [12]:
try:
    delete = transactions.delete_one(filter={'_id': insert.inserted_id})
    print('Number of documents deleted: {}'.format(delete.deleted_count))
    print('Visualize the raw result: ')
    pprint(delete.raw_result)
except Exception:
    print('Error during deletion.')

Number of documents deleted: 1
Visualize the raw result: 
{'$clusterTime': {'clusterTime': Timestamp(1652118464, 6),
                  'signature': {'hash': b'\xd5OS|\xc8\xcf\xed\xc0'
                                        b'\x99\x10\xc1\xa5\x99Om\x8a'
                                        b'\xa5\x04\x8e;',
                                'keyId': 7071822064142778372}},
 'electionId': ObjectId('7fffffff0000000000000006'),
 'n': 1,
 'ok': 1.0,
 'opTime': {'t': 6, 'ts': Timestamp(1652118464, 6)},
 'operationTime': Timestamp(1652118464, 6)}
